# Visualisierung eines Textkorpus mit Flair Embeddings, Pandas und Plotly

## Import

In [ ]:
import spacy
import string
import pickle

from umap import UMAP

import plotly.express as px
import pandas as pd



## Daten einlesen

In [ ]:
df = pd.read_pickle('../data/reden-bundesregierung.p')

In [ ]:
df.info()

In [ ]:
type(df.loc[0, 'doc_object'])

## Doc-Objekte in eine Liste

In [ ]:
docs = df.loc[:, 'doc_object'].to_list()

In [ ]:
type(docs[0])

In [ ]:
len(docs)

## Erstellen der gefilterten Matrix

In [ ]:
with open('../data/stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()

In [ ]:
matrix = list()
words = list()

In [ ]:
%%time

for doc in docs:
    
    for token in doc:
        if token.text is not token.is_punct \
        and token.text.lower() not in stopwords \
        and token.text.lower() not in words:
            embedding = token.tensor
            matrix.append(embedding)
            word = token.text.lower()
            words.append(word)
            
# statt token.text.lower() könnte für die Word-Liste auch token.lemma_.lower() genommen werden
# ferner NER der Worte mitnehmen, ggf. Tuples nutzen, die später wieder getrennt werden
# ferner POS der Worte mitnehmen, ggf. Tuples nutzen, die später wieder getrennt werden

In [ ]:
len(matrix)

In [ ]:
matrix[0]

In [ ]:
len(words)

## 3D-Scatterplot

In [ ]:
reducer_3d = UMAP(n_components=3, metric='cosine', n_neighbors=15, min_dist=0.1)

In [ ]:
%%time
reduced_matrix_3d = reducer_3d.fit_transform(matrix)

In [ ]:
reduced_matrix_3d.shape

In [ ]:
df_3d = pd.DataFrame.from_records(reduced_matrix_3d, columns=['x', 'y', 'z'])

In [ ]:
df_3d.loc[:, 'word'] = words

In [ ]:
df_3d.head()

In [ ]:
# save to file
df_3d.to_csv('../data/spacy-alle-worte-in-reden-umap-reduziert-3d-min-dist-01.csv', index=False)

In [ ]:
params = {'hover_data': {c: False for c in df_3d.columns},
          'hover_name': 'word'}

In [ ]:
fig = px.scatter_3d(df_3d, x='x', y='y', z='z', opacity=0.3, size_max=3, width=1000, height=1000, **params)
fig.show()
fig.write_html('../img/spacy-alle-worte-in-reden-umap-reduziert-3d-min-dist-01.html')

## Erstellen der gefilterten Matrix plus NER und POS

In [ ]:
with open('../data/stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()

In [ ]:
matrix_2 = list()
lemmas = list()
pos = list()
ner = list()

In [ ]:
%%time

for doc in docs:
    
    for token in doc:
        if token.text is not token.is_punct \
        and token.text.lower() not in stopwords \
        and token.lemma_.lower() not in lemmas:
            embedding = token.tensor
            matrix_2.append(embedding)
            lemma = token.lemma_.lower()
            lemmas.append(lemma)
            
            if token.ent_type_ in ['PER', 'ORG', 'LOC', 'MISC']:
                ner.append(token.ent_type_)
            else:
                ner.append('NaN')
            
            if token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                pos.append(token.pos_)
            else:
                pos.append('NaN')


In [ ]:
len(matrix_2),len(ner), len(lemmas), len(pos)

In [ ]:
ner[0], lemmas[0], pos[0]

In [ ]:
matrix_2[0]

## 3D-Scatterplot mit NER und POS

In [ ]:
reducer_3d = UMAP(n_components=3, metric='cosine', n_neighbors=15, min_dist=0.1)

In [ ]:
%%time
reduced_matrix_2_3d = reducer_3d.fit_transform(matrix_2)

In [ ]:
reduced_matrix_2_3d.shape

In [ ]:
df_ner_pos = pd.DataFrame.from_records(reduced_matrix_2_3d, columns=['x', 'y', 'z'])

In [ ]:
df_ner_pos.loc[:, 'lemma'] = lemmas

In [ ]:
df_ner_pos.loc[:, 'ner'] = ner

In [ ]:
df_ner_pos.loc[:, 'pos'] = pos

In [ ]:
df_ner_pos.head()

In [ ]:
# save to file
df_ner_pos.to_csv('../data/spacy-alle-worte-in-reden-umap-reduziert-3d-min-dist-01-ner-pos.csv', index=False)

## Plot Lemmas mit NER und POS

In [ ]:
params = {'hover_data': {c: False for c in df_ner_pos.columns},
          'hover_name': 'lemma'}

In [ ]:
fig = px.scatter_3d(df_ner_pos, x='x', y='y', z='z',
                    color='pos',
                 #   symbol='pos',
                    opacity=0.3, 
                    size_max=3, 
                    width=1000, 
                    height=1000, **params)
fig.show()
fig.write_html('../img/spacy-alle-worte-in-reden-umap-reduziert-3d-min-dist-01-pos.html')

In [ ]:
# plotly
# Color = 'ner'
# symbols = 'pos'